In [1]:
%pip install requests pandas



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def fetch_crypto_data(crypto_pair: str, start_date: str):
    # Split the crypto_pair into crypto_id and vs_currency
    crypto_id, vs_currency = crypto_pair.split('/')

    # Calculate the date 365 days ago
    one_year_ago = datetime.now() - timedelta(days=365)

    # Convert start date to datetime object
    start_date_dt = datetime.strptime(start_date, "%Y-%m-%d")

    # Use the later date of the provided start date or the 365 days ago date
    if start_date_dt < one_year_ago:
        start_date_dt = one_year_ago

    # Convert to Unix timestamp
    start_timestamp = int(start_date_dt.timestamp())
    end_timestamp = int((datetime.now() - timedelta(days=1)).timestamp())  # Till yesterday

    # URL for fetching market chart data
    url = f"https://api.coingecko.com/api/v3/coins/{crypto_id}/market_chart/range"
    params = {
        'vs_currency': vs_currency.lower(),  # Ensure the currency is in lowercase
        'from': start_timestamp,
        'to': end_timestamp
    }

    # Request data from CoinGecko
    response = requests.get(url, params=params)
    data = response.json()

    # Check if data was successfully retrieved
    if 'prices' not in data:
        print("Failed to retrieve data.")
        return pd.DataFrame()

    # Process data into DataFrame
    df = pd.DataFrame(data['prices'], columns=['Date', 'Close'])
    df['Open'] = df['Close'].shift(1).fillna(df['Close'])
    df['High'] = df['Close'].rolling(2).max()
    df['Low'] = df['Close'].rolling(2).min()
    df['Date'] = pd.to_datetime(df['Date'], unit='ms').dt.date
    df = df[['Date', 'Open', 'High', 'Low', 'Close']]

    return df

def save_to_csv(data: pd.DataFrame, filename: str):
    """Saves the DataFrame to a CSV file."""
    data.to_csv(filename, index=False)
    print(f"Data saved to {filename}")

if __name__ == "__main__":
    data = fetch_crypto_data("bitcoin/usd", "2023-01-01")  # Using BTC/USD as input
    if not data.empty:
        save_to_csv(data, "crypto_historical_data.csv")  
        print(data.head()) 


Data saved to crypto_historical_data.csv
         Date          Open          High           Low         Close
0  2023-10-30  34556.242815           NaN           NaN  34556.242815
1  2023-10-31  34556.242815  34556.242815  34498.703919  34498.703919
2  2023-11-01  34498.703919  34672.289284  34498.703919  34672.289284
3  2023-11-02  34672.289284  35457.454912  34672.289284  35457.454912
4  2023-11-03  35457.454912  35457.454912  34924.055450  34924.055450


In [4]:
import pandas as pd

def calculate_metrics(data: pd.DataFrame, variable1: int, variable2: int):
    # Ensure 'Date' column is a datetime object and set as index for easy rolling calculations
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)
    
    # Calculate Historical High Price and associated metrics
    data[f'High_Last_{variable1}_Days'] = data['High'].rolling(window=variable1).max()
    data[f'Days_Since_High_Last_{variable1}_Days'] = (data.index - data[data[f'High_Last_{variable1}_Days'].notnull()].index[-1]).days
    data[f'%_Diff_From_High_Last_{variable1}_Days'] = (data['Close'] - data[f'High_Last_{variable1}_Days']) / data[f'High_Last_{variable1}_Days'] * 100

    # Calculate Historical Low Price and associated metrics
    data[f'Low_Last_{variable1}_Days'] = data['Low'].rolling(window=variable1).min()
    data[f'Days_Since_Low_Last_{variable1}_Days'] = (data.index - data[data[f'Low_Last_{variable1}_Days'].notnull()].index[-1]).days
    data[f'%_Diff_From_Low_Last_{variable1}_Days'] = (data['Close'] - data[f'Low_Last_{variable1}_Days']) / data[f'Low_Last_{variable1}_Days'] * 100

    # Calculate Future High Price and % Difference
    future_highs = data['High'].shift(-variable2)
    data[f'High_Next_{variable2}_Days'] = future_highs.rolling(window=variable2).max()
    data[f'%_Diff_From_High_Next_{variable2}_Days'] = (data['Close'] - data[f'High_Next_{variable2}_Days']) / data[f'High_Next_{variable2}_Days'] * 100

    # Calculate Future Low Price and % Difference
    future_lows = data['Low'].shift(-variable2)
    data[f'Low_Next_{variable2}_Days'] = future_lows.rolling(window=variable2).min()
    data[f'%_Diff_From_Low_Next_{variable2}_Days'] = (data['Close'] - data[f'Low_Next_{variable2}_Days']) / data[f'Low_Next_{variable2}_Days'] * 100

    # Reset index to keep 'Date' as a column
    data.reset_index(inplace=True)
    
    return data

data = fetch_crypto_data("bitcoin/usd", "2023-01-01")
metrics_data = calculate_metrics(data, variable1=7, variable2=5)
print(metrics_data.head())


        Date          Open          High           Low         Close  \
0 2023-10-30  34556.242815           NaN           NaN  34556.242815   
1 2023-10-31  34556.242815  34556.242815  34498.703919  34498.703919   
2 2023-11-01  34498.703919  34672.289284  34498.703919  34672.289284   
3 2023-11-02  34672.289284  35457.454912  34672.289284  35457.454912   
4 2023-11-03  35457.454912  35457.454912  34924.055450  34924.055450   

   High_Last_7_Days  Days_Since_High_Last_7_Days  \
0               NaN                         -363   
1               NaN                         -362   
2               NaN                         -361   
3               NaN                         -360   
4               NaN                         -359   

   %_Diff_From_High_Last_7_Days  Low_Last_7_Days  Days_Since_Low_Last_7_Days  \
0                           NaN              NaN                        -363   
1                           NaN              NaN                        -362   
2             

In [5]:
metrics_data.to_csv("crypto_metrics_data.csv", index=False)  # Save to CSV without the index
print("Metrics data saved to 'crypto_metrics_data.csv'.")

Metrics data saved to 'crypto_metrics_data.csv'.
